# All imports

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from sklearn.gaussian_process import GaussianProcessRegressor 
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, WhiteKernel, ConstantKernel
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.metrics import pairwise_distances_argmin_min, mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import statsmodels.api as sm
import scipy as sc
from scipy.stats import multivariate_t, t, multivariate_normal
from scipy.special import loggamma
import sys

np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})

# Regression problem

We will use the diamonds dataset to explore Bayesian methods for regression, namely Gaussian Processes Regression, and Bayesian Linear Regression. We will aim to predict the prices of diamonds using the provided features.

## Dataset exploration

### Loading Dataset

In [ ]:
df = pd.read_csv('diamonds.csv')
df.head()

### Analysing dataset

In [ ]:
class Plotter():
    def __init__(self, data):
        self.data = data
        pass
    
    def plot(self, x, y, plt_type):
        if plt_type == 'scatter':
            plt.scatter(self.data[x], self.data[y])
            plt.title(f"{x} against {y}")
        if plt_type == 'box':
            sns.boxplot(data=self.data, x=x, y=y)
            plt.title(f"{x} against {y}")
        if plt_type == 'hist':
            sns.histplot(self.data[x], bins = 40, kde=False, edgecolor='black')
            plt.title(f"{x} Histogram")
            plt.ylabel('Frequency')
        plt.show()
 
pltr = Plotter(df)

#### Histogram plots of numerical features

We plot histograms to see which type of scaler would be appropriate while fitting models.

In [ ]:
pltr.plot('depth', None, 'hist')
pltr.plot('table', None, 'hist')
pltr.plot('carat', None, 'hist')
pltr.plot('price', None, 'hist')

#### Correlation Matrix of numerical features

We can see some highly correlated features including the group of z, x, y all being highly correlated with each other and with the carat covariate. We will remove these 3 covariates as a result of this, as it may inflate errors of regression coefficients, make them unstable, etc.

In [ ]:
corr = df.select_dtypes(include='number').corr()
plt.figure() 
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation of numerical features')
plt.show()

#### Scatter plot of low correlated features

In [ ]:
pltr.plot('depth', 'price', 'scatter')
pltr.plot('table', 'price', 'scatter')

#### Scatter plot of highly correlated features

In [ ]:
pltr.plot('carat', 'x', 'scatter')
pltr.plot('x', 'y', 'scatter')
pltr.plot('y', 'z', 'scatter')

#### Categorical/Ordinal feature analysis
Box and Whisker plots

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 4 * 1))
axes = axes.flatten()

sns.boxplot(data=df, x='color', y='price', ax=axes[0], order=['J', 'I', 'H', 'G', 'F', 'E', 'D'])
axes[0].set_title(f'Boxplot of color against price')
axes[0].set_xlabel('Color')
axes[0].set_ylabel('Price')

sns.boxplot(data=df, x='cut', y='price', ax=axes[1], order=['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'])
axes[1].set_title(f'Boxplot of cut against price')
axes[1].set_xlabel('Cut')
axes[1].set_ylabel('Price')

sns.boxplot(data=df, x='clarity', y='price', ax=axes[2], order=['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF'])
axes[2].set_title(f'Boxplot of clarity against price')
axes[2].set_xlabel('Clarity')
axes[2].set_ylabel('Price')

plt.tight_layout()
plt.show()

## Dataset Splitting

We split the dataset before preprocessing so we prevent data leakage.

In [ ]:
X = df.drop(columns=['price'])
y = df['price']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, shuffle=True, random_state=2)

## Dataset Preprocessing

### Preprocessing Classes

The DataCleaner class is used to clean the dataset by getting rid of duplicate rows, erroneous rows such as if a particular dimension is zero, and to drop features which are deemed unnecessary i.e. x, y, z.

Once cleaned, we can put the data through the preprocessing pipeline which encodes categorical variables and standardizes numerical variables.

In [ ]:
class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, subset=None, keep='first'):
        self.subset = subset
        self.keep = keep
        
    def reset_indices(self, X_cleaned, y_cleaned):
        return X_cleaned.reset_index(drop = True), y_cleaned.reset_index(drop = True)
    
    def get_list_of_similar_pairs(self, X):
        distance_matrix = euclidean_distances(X[['x', 'y', 'z']])
        threshold = 1e-3
        near_duplicate_pairs = np.argwhere(distance_matrix < threshold)
        filtered_pairs = [(i, j) for i, j in near_duplicate_pairs if i < j]
        
        rows_to_remove = set()
        for i,j in filtered_pairs:
            if i not in rows_to_remove and j not in rows_to_remove:
                rows_to_remove.add(j)
        
        return rows_to_remove

    def get_n_samples(self, X, n):
        kmeans = KMeans(n_clusters=n, random_state=42)
        kmeans.fit(X[['x', 'y', 'z', 'carat', 'depth', 'table']])
        indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X[['x', 'y', 'z', 'carat', 'depth', 'table']])
        return indices
        
    def filter(self, X_cleaned, y_cleaned, n):
        # Filter to get only 1500 data points as GP regression can't handle a lot of data
        indices_to_keep = self.get_n_samples(X_cleaned, n)
        X_cleaned = X_cleaned.iloc[indices_to_keep]
        y_cleaned = y_cleaned.iloc[indices_to_keep]
        X_cleaned, y_cleaned = self.reset_indices(X_cleaned, y_cleaned) 
        return X_cleaned, y_cleaned
        
    def remove_rows_with_faulty_diamond_size(self, X_cleaned, y_cleaned):
        mask_nonzero = ~((X_cleaned['x'] == 0) | (X_cleaned['y'] == 0) | (X_cleaned['z'] == 0))
        X_cleaned = X_cleaned[mask_nonzero]
        y_cleaned = y_cleaned.loc[X_cleaned.index]
        X_cleaned, y_cleaned = self.reset_indices(X_cleaned, y_cleaned) 
        return X_cleaned, y_cleaned
    
    def remove_duplicates(self, X, y):
        X_cleaned = X.drop_duplicates(subset=self.subset, keep=self.keep)
        y_cleaned = y.loc[X_cleaned.index]
        X_cleaned, y_cleaned = self.reset_indices(X_cleaned, y_cleaned) 
        return X_cleaned, y_cleaned
        
    def fit(self, X, y):
        return self

    def transform(self, X, y, filt = False, cols_to_drop = []):
        # Remove duplicate rows
        X_cleaned, y_cleaned = self.remove_duplicates(X, y)

        # Remove redundant data i.e. dimension of diamond is 0
        X_cleaned, y_cleaned = self.remove_rows_with_faulty_diamond_size(X_cleaned, y_cleaned)

        # Filter to get only 1500 items
        if filt:
            X_cleaned, y_cleaned = self.filter(X_cleaned, y_cleaned, 1500)
        
        # Drop certain features which are deemed unnecessary
        if len(cols_to_drop) > 0:
            X_cleaned = X_cleaned.drop(labels = cols_to_drop, axis = 1)
 
        return X_cleaned, y_cleaned
   
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y) 


ordinal_cols = ['cut', 'color', 'clarity']
ordinal_categories = [
    ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
    ['J', 'I', 'H', 'G', 'F', 'E', 'D'],
    ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
]
numeric_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']
numeric_cols = ['carat', 'depth', 'table']
numeric_cols_standardize = ['depth', 'table']
numeric_cols_normalize = ['carat']


preprocessor = ColumnTransformer(transformers=[
    ('ord', OrdinalEncoder(categories=ordinal_categories), ordinal_cols),
    ('num', RobustScaler(), numeric_cols)
], remainder='passthrough')


full_pipeline = Pipeline([
    ('preprocess', preprocessor),
])

#### Cleaning and Preprocessing the data

- 60% Training
- 20% Validation
- 20% Testing

In [ ]:
dc = DataCleaner()
X_train_cleaned, y_train_cleaned = dc.transform(X = X_train, y = y_train, cols_to_drop=['x','y','z'])
X_val_cleaned , y_val_cleaned = dc.transform(X = X_val, y = y_val, cols_to_drop=['x','y','z'])
X_test_cleaned , y_test_cleaned = dc.transform(X = X_test, y = y_test, cols_to_drop=['x','y','z'])

X_train_preprocessed = full_pipeline.fit_transform(X_train_cleaned, y_train_cleaned)
X_val_preprocessed = full_pipeline.fit_transform(X_val_cleaned, y_val_cleaned)
X_test_preprocessed = full_pipeline.fit_transform(X_test_cleaned, y_test_cleaned)

print("Training:")
print(X_train_preprocessed.shape)
print(y_train_cleaned.shape)
print("\nValidation")
print(X_val_preprocessed.shape)
print(y_val_cleaned.shape)
print("\nTesting:")
print(X_test_preprocessed.shape)
print(y_test_cleaned.shape)

## Model Training

### Model Selection functions

The below functions are explained when they are used below.

#### Random subsampling cross validation for Gaussian Process Regression

In [ ]:
def random_subsampling_split(X_train, y_train, tst_size = 2000, tr_size = 1500, seed = 1):
    X_temp, X_cross_val, y_temp, y_cross_val = train_test_split(X_train, y_train, test_size=tst_size, shuffle=True, random_state=seed)
    X_train_sub, _, y_train_sub, _ = train_test_split(X_temp, y_temp, train_size=tr_size, shuffle=True, random_state=seed)
    return X_train_sub, X_cross_val, y_train_sub, y_cross_val

'''
Takes in:
- X
- y
- full_pipeline
- scoring

Returns:
- Array of scores from cross validation using random subsampling

We use this for Gaussian Process Regression because it isn't possible for GP regression to be trained on all training samples due to computational complexity.
We train on a subset of the training set of 1500 samples. We still evaluate on the same number of validation samples in cross validation to maintain comparability of the cv score with other models.
'''
def random_subsampling_cv(full_pipeline, X_train_cleaned, y_train_cleaned, scoring, model, train_size = 1500, numeric_cols_only = False, n_splits=10):
    cv_scores = []
    
    X_train_preprocessed = full_pipeline.fit_transform(X_train_cleaned, y_train_cleaned)
    
    if numeric_cols_only:
        X_train_preprocessed = X_train_preprocessed[:, -3:]
    
    for i in range(n_splits):
        X_train_sub, X_val_sub, y_train_sub, y_val_sub = random_subsampling_split(
                X_train_preprocessed, 
                y_train_cleaned, 
                tst_size=(len(X_train_preprocessed) // n_splits),
                tr_size=train_size,
                seed=i)

        model.fit(X_train_sub, y_train_sub)

        y_pred_val_sub = model.predict(X_val_sub)
        score = scoring(y_val_sub, y_pred_val_sub)
        cv_scores.append(score)
        print("Finished split:", i+1)
    
    return cv_scores

#### K-Fold Cross Validation

In [ ]:
# Standard k-fold cross validation
def k_fold_cross_validation(full_pipeline, X_train_cleaned, y_train_cleaned, scoring, model, numeric_cols_only = False, n_splits=10):
    X_train_preprocessed = full_pipeline.fit_transform(X_train_cleaned, y_train_cleaned)
    if numeric_cols_only:
        X_train_preprocessed = X_train_preprocessed[:, -3:]
        
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_preprocessed, y_train_cleaned, cv = cv, scoring=scoring)
    return -scores

#### Bayesian Model Evidence Based Selection

In [ ]:
# Used for bayesian linear regression to select covariate set with highest evidence.
def bayesian_model_evidence_selection(full_pipeline, X_train_cleaned, y_train_cleaned, BayesianLinearRegressionClass, num_samples = 1000, num_models = 10, seed = 1):
    np.random.seed(seed)
    
    X_train_preprocessed = full_pipeline.fit_transform(X_train_cleaned, y_train_cleaned)
    
    param_indices = np.array([0,1,2,3,4,5])
    
    all_subsets = []
    all_evidences = []
    best_subset = []
    highest_evidence = -sys.maxsize - 1
    
    for i in range(num_models):
        subset_size = np.random.randint(1, len(param_indices))
        
        if i == 0:
            subset = [0,1,2,3,4,5]
        else:
            subset = np.random.choice(param_indices, subset_size, replace=False)
        subset = sorted(subset)
        
        blr = BayesianLinearRegressionClass(
                                mu_0=np.zeros(len(subset)+1),
                                a_0=0.01,
                                b_0=0.01,
                                fit_intercept=True,
                                unit_information_prior=True
                                )
        blr.fit(X_train_preprocessed[:num_samples,subset], y_train_cleaned[:num_samples])
        if blr.log_model_evidence > highest_evidence:
            highest_evidence = blr.log_model_evidence
            best_subset = subset
        
        all_subsets.append(subset)
        all_evidences.append(blr.log_model_evidence)
        
        print("Finished calculating evidence for subset number:", i+1)

    return highest_evidence, best_subset, all_evidences, all_subsets

### Model evaluation class
 This class is used to evaluate performance on a particular dataset, and takes in a fitted model. For example, evaluating performance on validation or test set and takes in fitted model on training set. It calculates all sorts of regression metrics and plots, comprehensively evaluating performance.

In [ ]:
# Classes used to evaluate fitted regressors using a range of evaluation metrics.
class RegressionEvaluator(ABC):
    def __init__(self, model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names):
        self.model = model
        self.feature_names = feature_names
        self.real = y_evaluation_cleaned 
        self.preds = self.model.predict(X_evaluation_preprocessed[:, :])

    @abstractmethod
    def plot_coefficients(self):
        pass
    
    @abstractmethod
    def get_coefficient_statistics(self):
        pass
    
    def get_metrics(self):
        mse = mean_squared_error(self.real, self.preds)
        mae = mean_absolute_error(self.real, self.preds)
        r2_coefficient = r2_score(self.real, self.preds)
        return {
            'mse': mse,
            'mae': mae,
            'r2_coefficient': r2_coefficient
            }

            
    def plot_regression_plot(self):
        sns.regplot(x = self.real, y = self.preds, color='blue', scatter_kws={'alpha': 0.5}, line_kws={'color': 'maroon'})
        plt.plot([self.real.min(), self.real.max()], [self.real.min(), self.real.max()], 'k--', lw=2)
        plt.title('True Values against Predicted Values')
        plt.xlabel('True values', fontsize=14)
        plt.ylabel('Predicted values', fontsize=14)

        legend_elements = [
            Line2D([0], [0], marker='o', color='w', label='Data', markerfacecolor='blue', markersize=8, alpha=0.5),
            Line2D([0], [0], color='maroon', lw=2, label='Regression Line'),
            Line2D([0], [0], color='k', lw=2, linestyle='--', label='45° Line (Ideal)')
        ]

        plt.legend(handles = legend_elements)
        plt.show()

    def plot_residual_plot(self):
        residuals = self.real - self.preds
        sns.residplot(x=self.preds, y=residuals, color='blue', scatter_kws={'alpha':0.5})
        plt.title('Residuals Plot')
        plt.xlabel('Predicted values', fontsize=14)
        plt.ylabel('Residuals', fontsize=14)
        plt.show()

    def get_all_regression_metrics(self):
        metrics = self.get_metrics()
        print(metrics)
        self.plot_coefficients()
        self.plot_regression_plot()
        self.plot_residual_plot()
        self.get_coefficient_statistics()

        
class BayesianLinearRegressionEvaluator(RegressionEvaluator):
    def __init__(self, model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names):
        super().__init__(model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names)
        
    def plot_coefficients(self):
        coefficients = self.model.coef_
        self.feature_names = pd.Index(['intercept']).append(self.feature_names)
        plt.bar(range(len(self.feature_names)), np.abs(coefficients))
        plt.xticks(range(len(self.feature_names)),self.feature_names[:len(self.feature_names)],rotation=-45)
        plt.xlabel('Coefficient')
        plt.ylabel('Absolute value - log scale')
        plt.yscale('log')
        plt.show()
    
    def get_coefficient_statistics(self):
        pd.set_option('display.width', None)
        pd.set_option('display.max_columns', None)
        print(self.model.get_summary().to_string())
        print()
        

class LinearRegressionEvaluator(RegressionEvaluator):
    def __init__(self, model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names, stats_model):
        self.stats_model = stats_model
        super().__init__(model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names)
        
    def plot_coefficients(self):
        coefficients = self.model.coef_
        plt.bar(range(len(self.feature_names)), np.abs(coefficients))
        plt.xticks(range(len(self.feature_names)),self.feature_names[:len(self.feature_names)],rotation=-45)
        plt.xlabel('Coefficient')
        plt.ylabel('Absolute value - log scale')
        plt.yscale('log')
        plt.show()
        
    def get_coefficient_statistics(self):
        results = self.stats_model.fit()
        print(results.summary())
        print()
    
class GaussianProcessRegressionEvaluator(RegressionEvaluator):
    def __init__(self, model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names):
        super().__init__(model, X_evaluation_preprocessed, y_evaluation_cleaned, feature_names)
        
    def _plot_coefficient_plotter(self, importance):
        plt.bar(range(len(self.feature_names)), np.abs(importance))
        plt.xticks(range(len(self.feature_names)),self.feature_names[:len(self.feature_names)],rotation=-45)
        plt.title("GP Feature Importance (1 / Length Scale)")
        plt.xlabel('Coefficient')
        plt.ylabel('Feature Importance (log scale)')
        plt.yscale('log')
        plt.show()
        
    def get_coefficient_statistics(self):
        return super().get_coefficient_statistics()
    
    def plot_coefficients(self):
        kernel_params = self.model.kernel_.get_params()
        length_scale_params = []
        for kp_key in kernel_params:
            if kp_key.endswith("length_scale"):
                length_scale_params.append(kp_key)
        
        for i in range(len(length_scale_params)):
            coefficients = kernel_params[length_scale_params[i]]
            if isinstance(coefficients, np.float64):
                print("Isotropic kernel with length_scale value:", coefficients)
                continue
            importance = 1 / coefficients
            self._plot_coefficient_plotter(importance)

### Gaussian Process Regression

#### Gaussian Process Regression using 1 covariate: Plotting the fit

Here, we are training a basic Gaussian Process regression model using quadratic exponential kernel on just 1 covariate (carat). We do this so we can visually see how Gaussian Process Regression actually works.
This section serves as a foundation for later more complex Gaussian Process models, which will utilize all covariates. We will not be able to visualize the fits of the regression in those cases.
We will also plot 95% confidence intervals across the fit.

In [ ]:
kernel = ConstantKernel(constant_value=1,constant_value_bounds=(1e+1, 1e+10)) * RBF(length_scale=1.0, length_scale_bounds=(1e-5, 100)) + WhiteKernel(noise_level=1 ** 2, noise_level_bounds=(1e-5, 1e+10))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)


X_train_preprocessed_1500, _, y_train_cleaned_1500, _ = random_subsampling_split(X_train_preprocessed, y_train_cleaned, tr_size = 1500)
gp.fit(X_train_preprocessed_1500[:, -3:-2], y_train_cleaned_1500)
gp.kernel_

In [ ]:
# For the val set
y_preds_val_set, sigma_preds_val_set= gp.predict(
    X_val_preprocessed[:, -3:-2], 
    return_std=True
    )

# For all values in a linear space, best option for plotting.
X_linspace = np.expand_dims(np.linspace(-10, 10, 10000), 1)
y_preds_linspace, sigma_preds_linspace= gp.predict(
    X_linspace, 
    return_std=True
    )

In [ ]:
mse = mean_squared_error(y_preds_val_set, y_val_cleaned)
mae = mean_absolute_error(y_preds_val_set, y_val_cleaned)
r2 = r2_score(y_preds_val_set, y_val_cleaned)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R2 coefficient", r2)

plt.plot(X_train_preprocessed_1500[:, -3], y_train_cleaned_1500, 'o', label = 'Data')
plt.plot(X_val_preprocessed[:, -3], y_val_cleaned, 'yo', label='Validation Observations')
plt.plot(X_linspace, y_preds_linspace)
plt.fill(
    np.concatenate([X_linspace, X_linspace[::-1]]),
    np.concatenate(
        [y_preds_linspace - 1.9600 * sigma_preds_linspace,
        (y_preds_linspace + 1.9600 * sigma_preds_linspace)[::-1]]),
    alpha=.5, fc='b', ec='None', label='95% prediction interval'
)
plt.legend(loc='upper left')
plt.show()

#### Cross-validation to find best Gaussian Process Kernel

Here we will do cross-validation via subsampling within the training set to find the best Gaussian Process model. The Gaussian Process model is fitted on all covariates. We then take the model with the lowest average MSE across all splits (10 splits). The reason we do the subsampling approach instead of k-fold approach is because Gaussian Process models are prohibited by the number of data points due to the algorithm's computational time complexity. We sample 1500 points from the training split, and then validate on unseen validation split in each fold.

NOTE: I tried many more models, but to save time and space have presented results of only 2 models. The best results were achieved by the isotropic RBF kernel

Note: Below cell took ~20 mins to run on Apple MacBook M1 Air

In [ ]:
kernel = ConstantKernel(constant_value=1,constant_value_bounds=(1e+1, 1e+10)) * RBF(length_scale=1.0, length_scale_bounds=(1e-5, 100)) + WhiteKernel(noise_level=1 ** 2, noise_level_bounds=(1e-5, 1e+10))
gp_rbf = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

cv_scores_gp_rbf_all_cols = random_subsampling_cv(full_pipeline, X_train_cleaned, y_train_cleaned, mean_squared_error, gp_rbf, train_size = 1500, numeric_cols_only=False)
print(cv_scores_gp_rbf_all_cols)
print(np.mean(cv_scores_gp_rbf_all_cols))

Note: Below cell took ~40 mins to run on Apple MacBook M1 Air

In [ ]:
kernel = ConstantKernel(constant_value=1,constant_value_bounds=(1e+1, 1e+10)) * RationalQuadratic(length_scale=1.0, alpha=1.0) + WhiteKernel(noise_level=1 ** 2, noise_level_bounds=(1e-5, 1e+10))
gp_rq = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

cv_scores_gp_rq_all_cols = random_subsampling_cv(full_pipeline, X_train_cleaned, y_train_cleaned, mean_squared_error, gp_rq, train_size = 1500, numeric_cols_only=False)
print(cv_scores_gp_rq_all_cols)
print(np.mean(cv_scores_gp_rq_all_cols))

#### Refitting best Gaussian Process model on entire training set and seeing performance on Validation Set

##### Refitting Gaussian on entire training set and evaluating fit

In [ ]:
kernel = ConstantKernel(constant_value=1,constant_value_bounds=(1e+1, 1e+10)) * RBF(length_scale=1.0, length_scale_bounds=(1e-5, 1e+5)) + WhiteKernel(noise_level=1 ** 2, noise_level_bounds=(1e-5, 1e+10))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
X_train_preprocessed_1500, _, y_train_cleaned_1500, _ = random_subsampling_split(X_train_preprocessed, y_train_cleaned, tr_size = 1500)
gp.fit(X_train_preprocessed_1500[:, :], y_train_cleaned_1500)

In [ ]:
gp.kernel_

In [ ]:
regression_evaluator_gp = GaussianProcessRegressionEvaluator(model=gp, X_evaluation_preprocessed=X_val_preprocessed, y_evaluation_cleaned=y_val_cleaned, feature_names = X_train_cleaned.columns)
regression_evaluator_gp.get_all_regression_metrics()

### Linear regression

#### Cross validation
In Standard Linear Regression, we aren't tuning any hyperparameters apart from choosing the amount of covariates. Removing any single covariate resulted in a loss of performance according to MSE. In addition, all coefficients are statistically significant according to t-tests conducted by the statsmodel package (this is done as part of the evaluator class).

I have shown the cross-validation results below of the model with all covariates

In [ ]:
cv_scores_lr_all_cols = k_fold_cross_validation(full_pipeline, X_train_cleaned, y_train_cleaned, 'neg_mean_squared_error', LinearRegression(), numeric_cols_only=False, n_splits=10)
print(cv_scores_lr_all_cols)
print(np.mean(cv_scores_lr_all_cols))

#### Refitting Linear Regression model on entire training set and seeing performance on Validation Set

In [ ]:
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train_preprocessed[:, :], y_train_cleaned)

In [ ]:
regression_evaluator_lr = LinearRegressionEvaluator(model=lr, 
                                                    X_evaluation_preprocessed=X_val_preprocessed, 
                                                    y_evaluation_cleaned=y_val_cleaned, 
                                                    feature_names = X_train_cleaned.columns,
                                                    stats_model=sm.OLS(y_val_cleaned, sm.add_constant(X_val_preprocessed[:, :])))
regression_evaluator_lr.get_all_regression_metrics()

### Bayesian Linear Regression

The following class implements Bayesian Linear Regression using a unit information prior and exposes a class interface similar to sklearn's implementation of regressors. It inherits from the base regressor class part of the sklearn package. It extends from it as after fitting, it is able to provide a pandas dataframe of the summary of the fitted model including coefficients and 95% confidence intervals for parameter estimates. This 95% confidence interval is constructed from both Monte-Carlo method and using a t-distribution allowing me to compare. It also automatically calculates model evidence after fitting to data.

In [ ]:
class BayesianLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self, omega_0 = [[1.0]], mu_0 = [0], a_0 = 0.01, b_0 = 0.01, fit_intercept = True, unit_information_prior = True):
        self.omega_0 = omega_0 
        self.omega_0_inv = sc.linalg.inv(self.omega_0)
        self.mu_0 = mu_0
        self.a_0 = a_0
        self.b_0 = b_0
        self.fit_intercept = fit_intercept
        self.unit_information_prior = unit_information_prior

        
    def _calculate_posteriors(self, X, y):
        if self.fit_intercept:
            X = np.column_stack([np.ones(X.shape[0]), X])
        
        XtX = X.T @ X
        n, p = X.shape

        if self.unit_information_prior:
            self.omega_0 = n * sc.linalg.inv(X.T @ X)
            self.omega_0_inv = sc.linalg.inv(self.omega_0)
        
        self.omega_n = sc.linalg.inv(XtX + self.omega_0_inv)
        self.mu_n = self.omega_n @ (self.omega_0_inv @ self.mu_0 + X.T @ y)
        self.a_n = self.a_0 + n/2
        
        term2 = y.T @ y + self.mu_0.T @ self.omega_0_inv @ self.mu_0 + self.mu_n.T @ sc.linalg.inv(self.omega_n) @ self.mu_n
        self.b_n = self.b_0 + term2/2

    def _get_marginal_t_distribution(self, mu, Sigma, j, nu):
        sigma_j2 = Sigma[j, j]
        return t(df = nu, loc=mu[j], scale = np.sqrt(sigma_j2))
    
    # y ~ N(XB, sigma^2 * I_n)
    # p(y|B, sigma^2, X)
    def __calculate_log_likelihood(self, X, y):
        n, p = X.shape
        # The variance of the residuals is distributed according to inverse gamma distribution
        # The posterior params are a_n, b_n
        sigma2 = self.b_n / (self.a_n - 1) # The mean of inverse gamma with params a, b is b / (a - 1)
        cov = sigma2 * np.eye(n)
        beta = self.mu_n
        mu = X @ beta
        log_likelihood = multivariate_normal.logpdf(y,mu,cov) # Note this line will struggle to when n > 10,000 due to 10,000x10,000 covariance matrix
        return log_likelihood
    
    def __calculate_log_prior(self):
        sigma2 = self.b_n / (self.a_n - 1)
        beta = self.mu_n
        log_prior = self.a_0 * np.log(self.b_0) - loggamma(self.a_0) - (self.a_0 + 1) * np.log(sigma2) - self.b_0 / sigma2
        log_prior = log_prior + multivariate_normal.logpdf(beta,self.mu_0,sigma2*self.omega_0)
        return log_prior
    
    def __calculate_log_posterior(self):
        sigma2 = self.b_n / (self.a_n - 1)
        beta = self.mu_n
        log_posterior = self.a_n * np.log(self.b_n) - loggamma(self.a_n) - (self.a_n + 1) * np.log(sigma2) - self.b_n / sigma2
        log_posterior = log_posterior + multivariate_normal.logpdf(beta,self.mu_n,sigma2*self.omega_n)
        return log_posterior
         
    def _calculate_model_evidence(self, X, y):
        if self.fit_intercept:
            X = np.column_stack([np.ones(X.shape[0]), X])
        log_likelihood = self.__calculate_log_likelihood(X, y)
        log_prior = self.__calculate_log_prior()
        log_posterior = self.__calculate_log_posterior()
        log_model_evidence = log_likelihood + log_prior - log_posterior
        return log_model_evidence   
    
    
    def get_summary(self):
        check_is_fitted(self)
        
        scale = (self.b_n / self.a_n) * self.omega_n
        lower5_t = []
        upper5_t = []
        for i in range(0, len(scale)):
            marginal = self._get_marginal_t_distribution(self.mu_n, scale, i, 2 * self.a_n)
            lower, upper = marginal.ppf([0.025, 0.975])
            lower5_t.append(lower)
            upper5_t.append(upper)
        lower5_t = np.array(lower5_t)
        upper5_t = np.array(upper5_t)

        np.random.seed(1)
        betas = multivariate_t.rvs(loc = self.mu_n, shape=scale, df=2 * self.a_n, size=10000000)
        lower5_sampling = np.percentile(betas, 2.5, axis=0)
        upper5_sampling = np.percentile(betas, 97.5, axis=0)
        results = np.column_stack([self.mu_n, lower5_sampling, upper5_sampling, lower5_t, upper5_t])
        summary = pd.DataFrame(results, columns=['posterior mean','lower 95% bound (Monte Carlo Sampling)',
                                                 'upper 95% bound (Monte Carlo Sampling)',
                                                 'lower 95% bound (t-dist value)',
                                                 'upper 95% bound (t-dist value)'], 
                               index=[f'x{j+1}' for j in range(len(scale))])
        return summary
        

    def fit(self, X, y):
        X, y = check_X_y(X, y)
            
        self._calculate_posteriors(X, y)
        self.coef_ = self.mu_n
        self.log_model_evidence = self._calculate_model_evidence(X, y)
        return self
        
        
    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        if self.fit_intercept:
            X = np.column_stack([np.ones(X.shape[0]), X])
        return X @ self.mu_n

#### Bayesian Model Evidence
For the Bayesian Linear Regression model, I decided to choose the number of covariates according to the highest marginal likelihood. I used the unit information prior with mean 0. This way we automatically choose a model while optimizing parsimony as opposed to optimizing predictive power, and seeing if this approach is able to significantly reduce the error on the validation set using this alternative approach.

One thing to note is calculating the likelihood is computationally extremely difficult when $n \geq 10000$ (The covariance matrix is of this dimension), and this is necessary to calculate the marginal likelihood. Hence we are limited to training on a subset of the training dataset of size 10,000.

NOTE: The below cell took 30 mins to run


In [ ]:

best_evidence, best_feature_subset, all_evidences, all_feature_subsets = bayesian_model_evidence_selection(full_pipeline, 
                                                                            X_train_cleaned, 
                                                                            y_train_cleaned, 
                                                                            BayesianLinearRegression,
                                                                            num_samples=10000,
                                                                            num_models=10,
                                                                            )

print("Best Model Evidence:", best_evidence)
print("Best Feature Subset:", best_feature_subset)

In [ ]:
print(all_evidences)
print(all_feature_subsets)

#### Refitting model with highest marginal likelihood on the training set and evaluating performance on validation

In [ ]:
blr = BayesianLinearRegression(
                               mu_0=np.zeros(len(best_feature_subset)+1),
                               a_0=0.01,
                               b_0=0.01,
                               fit_intercept=True,
                               unit_information_prior=True
                               )

num_samples_blr = 10000
blr.fit(X_train_preprocessed[:num_samples_blr,best_feature_subset], y_train_cleaned[:num_samples_blr])

In [ ]:
regression_evaluator_blr = BayesianLinearRegressionEvaluator(model=blr, 
                                                    X_evaluation_preprocessed=X_val_preprocessed[:, best_feature_subset], 
                                                    y_evaluation_cleaned=y_val_cleaned, 
                                                    feature_names = X_train_cleaned.columns[best_feature_subset],
                                                    )
regression_evaluator_blr.get_all_regression_metrics()

## Final Model Evaluation

The best model we found was the isotropic Gaussian Process Regressor. This gave a validation MSE of around 540,000. The error on the test set is lower than the validation, of around 510,000. This model marks a signicant improvement compared to Linear Regression and Bayesian Linear Regression models, highlighting the non-linear structure of the underlying data. This has successfully been captured by a Gaussian Process Regressor

In [ ]:
regression_evaluator_gp_test_set = GaussianProcessRegressionEvaluator(model=gp, X_evaluation_preprocessed=X_test_preprocessed, y_evaluation_cleaned=y_test_cleaned, feature_names = X_train_cleaned.columns)
regression_evaluator_gp_test_set.get_all_regression_metrics()